In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%cd /content/drive/My Drive

/content/drive/My Drive


In [1]:
# %ls

This is the take-home assignment for ML Audio role at Alethea AI.

You will be using this repo https://github.com/yl4579/StyleTTS in this assignment. This is a style-based generative model for parallel TTS that can synthesize diverse speech with natural prosody from a reference speech utterance. First, please take a look at the README to get a high-level overview of the model, the inputs, outputs and how it works. Then go through this notebook and complete the provided requirements (please work on a copy this colab and send it to us when you completed the assignment.)

NOTE: please choose a GPU-enabled runtime when running this colab.

Task: Voice cloning
Given the recordings of a speaker, we want to produce the synthetic voice with high level of naturalness and acoustic quality, mimicking the nuances of the target speaker. You will be using StyleTTS to perform the voice cloning task. First, please fork and push the original StyleTTS repo to your personal repo. (remember to share it with these users https://github.com/ahmedjawedaj https://github.com/sinhprous1)

Now, please clone your personal StyleTTS repo.

In [4]:
# !git clone https://github.com/Xrehman/StyleTTS.git
!cd StyleTTS


In [5]:
!pip install SoundFile torchaudio munch torch pydub pyyaml librosa git+https://github.com/resemble-ai/monotonic_align.git

  Cloning https://github.com/resemble-ai/monotonic_align.git to /tmp/pip-req-build-nhb7kn36
  Running command git clone --filter=blob:none --quiet https://github.com/resemble-ai/monotonic_align.git /tmp/pip-req-build-nhb7kn36
  Resolved https://github.com/resemble-ai/monotonic_align.git to commit 78b985be210a03d08bc3acc01c4df0442105366f
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [6]:
!pip install librosa==0.8.1 attrdict

In [7]:
!apt install -y espeak
!pip install phonemizer

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
espeak is already the newest version (1.48.15+dfsg-3).
0 upgraded, 0 newly installed, 0 to remove and 19 not upgraded.


This is the link to the provided dataset that consists of the recordings and the data list file (labels.txt). https://drive.google.com/drive/folders/1RU09QolgQpt-w1labrT7cKF3aNpsR2In

Please upload it to your Google Drive and mount drive to colab. Then please write the code to convert labels.txt file to the correct format (see example in the original repo: https://github.com/yl4579/StyleTTS/blob/main/Data/val_list_libritts.txt) and push the output data list file to your personal github repo, then share the link here.

Furthermore, the sampling rate of audios is 16khz which is not compatible to StyleTTS training code, please also resample the audios here so that they can be trained by StyleTTS model.

NOTE: please upload any file that you need to Google drive and use it by mounted path, since uploading files directly in colab is extremely slow.

In [8]:
%cd /content/drive/MyDrive/StyleTTS/

/content/drive/MyDrive/StyleTTS


In [12]:
# code for converting the data to the right format
# load phonemizer
# i have done this my on local PC and shift to Colab after that when cuda OOM.
import phonemizer
global_phonemizer = phonemizer.backend.EspeakBackend(language='en-us', preserve_punctuation=True,  with_stress=True)
def phonemize(global_phonemizer, text):
    return global_phonemizer.phonemize([text])[0]
file_name = r'C:\Users\rehma\OneDrive\Desktop\tts\StyleTTS\dataset\labels.txt'

new_lines = []
with open(file_name, 'r') as file:
    # Read and process each line
    for line in file:
        # Do something with each line, e.g., print it
        phns = line.split('|')[-1].replace('\n', '')
        phns = phonemize(global_phonemizer, phns)
        new_lines.append( line.split('|')[0] + '|' + phns)

with open(file_name.replace('labels.txt', 'final_labels.txt'), 'w', encoding='utf-8') as file:
    # Write each line to the file
    for line in new_lines:
        file.write(line + '\n')
# dataset spliting
import random

# Set the paths for the input training file and the output training and validation files
input_file = r'C:\Users\rehma\OneDrive\Desktop\tts\StyleTTS\dataset\final_labels.txt'
output_train_file = r'C:\Users\rehma\OneDrive\Desktop\tts\StyleTTS\dataset\train.txt'
output_validation_file = r'C:\Users\rehma\OneDrive\Desktop\tts\StyleTTS\dataset\val.txt'

# Define the ratio for the split (e.g., 80% training, 20% validation)
split_ratio = 0.8

# Read the lines from the input training file
with open(input_file, 'r', encoding='utf-8') as f:
    lines = f.readlines()

# Shuffle the lines randomly
random.shuffle(lines)

# Calculate the split index
split_index = int(len(lines) * split_ratio)

# Split the lines into training and validation sets
training_set = lines[:split_index]
validation_set = lines[split_index:]

# Write the training set to the output training file
with open(output_train_file, 'w', encoding='utf-8') as f:
    f.writelines(training_set)

# Write the validation set to the output validation file
with open(output_validation_file,'w', encoding='utf-8') as f:
    f.writelines(validation_set)

print(f"Training and validation sets are saved to {output_train_file} and {output_validation_file}.")

# also please resample the audios


import os
import librosa
import soundfile as sf

# Set the source and target directories
source_folder = r'C:\Users\rehma\OneDrive\Desktop\tts\StyleTTS\dataset\wavs'
target_folder = target_folder =  r'C:\Users\rehma\OneDrive\Desktop\tts\StyleTTS\dataset\final_wavs'

# Create the target folder if it doesn't exist
os.makedirs(target_folder, exist_ok=True)

# Function to upsample audio and save it to the target folder
def upsample_and_save(input_path, output_path):
    y, sr = librosa.load(input_path, sr=16000)
    y_resampled = librosa.resample(y, orig_sr=16000, target_sr=24000)
    sf.write(output_path, y_resampled, 24000)
    print(f'Upsampled and saved: {output_path}')

# Traverse the source folder and upsample audio files
for root, dirs, files in os.walk(source_folder):
    for filename in files:
        if filename.endswith('.wav'):
            input_path = os.path.join(root, filename)
            output_path = os.path.join(target_folder, filename)
            upsample_and_save(input_path, output_path)

print('All audio files upsampled and saved to the target folder.')
# note that you also need to split the data into training and validation splits.


In [ ]:
# link to final process labels text file is https://github.com/Xrehman/StyleTTS/blob/main/TTS-Assignment/final_labels.txt


This is the default configuration file for training StyleTTS https://github.com/yl4579/StyleTTS/blob/main/Configs/config.yml . Please prepare your own file for training the provided dataset and share the link to your configuration file here. You need to change some important parameters (e.g: pretrained model path, log_dir, epochs, batch size, training data, validation data ... ).

NOTE: it takes around 1 - 1.5 hours in colab (T4 GPU) to train the model with epochs_1st and epochs_2nd are set to 61, save_freq is 20.

In [ ]:
# link to configuration file https://github.com/Xrehman/StyleTTS/blob/main/TTS-Assignment/config.yml.txt

In [73]:
!python train_first.py --config_path Configs/config.yml

2023-11-03 01:07:12.453023: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-03 01:07:12.453091: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-03 01:07:12.453134: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-03 01:07:12.464948: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-03 01:07:14.278909: W tensorflow/c

In [9]:
!pwd

/content/drive/MyDrive/StyleTTS


In [20]:
!pip install SoundFile==0.10.3.post1

  Attempting uninstall: SoundFile
    Found existing installation: soundfile 0.12.1
    Uninstalling soundfile-0.12.1:
      Successfully uninstalled soundfile-0.12.1


In [23]:
from numba import cuda
device = cuda.get_current_device()
device.reset()

In [25]:
!python train_second.py --config_path Configs/config.yml

2023-11-03 02:25:25.117030: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-03 02:25:25.117093: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-03 02:25:25.117131: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-03 02:25:25.125180: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-03 02:25:26.232262: W tensorflow/c

After training is completed. Please try generating some samples using the following cells.

In [9]:
# load packages
import random
import yaml
from munch import Munch
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F
import torchaudio
import librosa

from models import *
from utils import *

%matplotlib inline

In [10]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
_pad = "$"
_punctuation = ';:,.!?¡¿—…"«»“” '
_letters = 'ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz'
_letters_ipa = "ɑɐɒæɓʙβɔɕçɗɖðʤəɘɚɛɜɝɞɟʄɡɠɢʛɦɧħɥʜɨɪʝɭɬɫɮʟɱɯɰŋɳɲɴøɵɸθœɶʘɹɺɾɻʀʁɽʂʃʈʧʉʊʋⱱʌɣɤʍχʎʏʑʐʒʔʡʕʢǀǁǂǃˈˌːˑʼʴʰʱʲʷˠˤ˞↓↑→↗↘'̩'ᵻ"


# Export all symbols:
symbols = [_pad] + list(_punctuation) + list(_letters) + list(_letters_ipa)

dicts = {}
for i in range(len((symbols))):
    dicts[symbols[i]] = i

class TextCleaner:
    def __init__(self, dummy=None):
        self.word_index_dictionary = dicts
    def __call__(self, text):
        indexes = []
        for char in text:
            try:
                indexes.append(self.word_index_dictionary[char])
            except KeyError:
                print(char)
        return indexes

textclenaer = TextCleaner()

In [11]:
to_mel = torchaudio.transforms.MelSpectrogram(
    n_mels=80, n_fft=2048, win_length=1200, hop_length=300)
mean, std = -4, 4

def length_to_mask(lengths):
    mask = torch.arange(lengths.max()).unsqueeze(0).expand(lengths.shape[0], -1).type_as(lengths)
    mask = torch.gt(mask+1, lengths.unsqueeze(1))
    return mask

def preprocess(wave):
    wave_tensor = torch.from_numpy(wave).float()
    mel_tensor = to_mel(wave_tensor)
    mel_tensor = (torch.log(1e-5 + mel_tensor.unsqueeze(0)) - mean) / std
    return mel_tensor

def compute_style(ref_dicts):
    reference_embeddings = {}
    for key, path in ref_dicts.items():
        print(key)
        refs = []
        if type(path) == str:
            paths = [path]
        else:
            paths = path
        audios = []
        for path in paths:
            wave, sr = librosa.load(path, sr=24000)
            audio, index = librosa.effects.trim(wave, top_db=30)
            if sr != 24000:
                audio = librosa.resample(audio, sr, 24000)
            audios.append(audio)
            mel_tensor = preprocess(audio).to(device)
            try:
                with torch.no_grad():
                    ref = model.style_encoder(mel_tensor.unsqueeze(1))
                    refs.append(ref)
                    # print(ref.shape)
            except Exception as e:
                print(e)
                continue
        if len(refs) == 0:
            raise ValueError
        reference_embeddings[key] = (torch.mean(torch.cat(refs,axis=0),axis=0).unsqueeze(0), audios)

    return reference_embeddings

In [12]:
# load phonemizer
import phonemizer
global_phonemizer = phonemizer.backend.EspeakBackend(language='en-us', preserve_punctuation=True,  with_stress=True)

In [13]:
!pwd

/content/drive/MyDrive/StyleTTS


In [14]:
# load hifi-gan
# Monkey patch collections
# un-comment this code if you face problem with collections
"""
import collections
import collections.abc
for type_name in collections.abc.__all__:
    setattr(collections, type_name, getattr(collections.abc, type_name))
"""

import sys
sys.path.insert(0, "./Demo/hifi-gan")

import glob
import os
import argparse
import json
import torch
from scipy.io.wavfile import write
from attrdict import AttrDict
from vocoder import Generator
import librosa
import numpy as np
import torchaudio

h = None

def load_checkpoint(filepath, device):
    assert os.path.isfile(filepath)
    print("Loading '{}'".format(filepath))
    checkpoint_dict = torch.load(filepath, map_location=device)
    print("Complete.")
    return checkpoint_dict

def scan_checkpoint(cp_dir, prefix):
    pattern = os.path.join(cp_dir, prefix + '*')
    cp_list = glob.glob(pattern)
    if len(cp_list) == 0:
        return ''
    return sorted(cp_list)[-1]

cp_g = scan_checkpoint("Vocoder/LibriTTS/", 'g_')
print(os.path.split(cp_g)[0])
config_file = os.path.join(os.path.split(cp_g)[0], 'config.json')
with open(config_file) as f:
    data = f.read()
json_config = json.loads(data)
h = AttrDict(json_config)

device = torch.device(device)
generator = Generator(h).to(device)

state_dict_g = load_checkpoint(cp_g, device)
generator.load_state_dict(state_dict_g['generator'])
generator.eval()
generator.remove_weight_norm()

Vocoder/LibriTTS


/usr/local/lib/python3.10/dist-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


Loading 'Vocoder/LibriTTS/g_00750000'
Complete.
Removing weight norm...


In [15]:
# load StyleTTS
model_path = "/content/drive/MyDrive/StyleTTS/Models/ttsAssignment/epoch_2nd_00060.pth" # use your trained model here
model_config_path = "/content/drive/MyDrive/StyleTTS/Models/ttsAssignment/config.yml" # use your configuration file here

config = yaml.safe_load(open(model_config_path))

# load pretrained ASR model
ASR_config = config.get('ASR_config', False)
ASR_path = config.get('ASR_path', False)
text_aligner = load_ASR_models(ASR_path, ASR_config)

# load pretrained F0 model
F0_path = config.get('F0_path', False)
pitch_extractor = load_F0_models(F0_path)

model = build_model(Munch(config['model_params']), text_aligner, pitch_extractor)

params = torch.load(model_path, map_location='cpu')
params = params['net']
for key in model:
    if key in params:
        if not "discriminator" in key:
            print('%s loaded' % key)
            model[key].load_state_dict(params[key])
_ = [model[key].eval() for key in model]
_ = [model[key].to(device) for key in model]

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/rnn.py:82: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


predictor loaded
decoder loaded
pitch_extractor loaded
text_encoder loaded
style_encoder loaded
text_aligner loaded


In [16]:
reference_audio_path = ['/content/drive/MyDrive/Data/wavs/anger_367-392_0375.wav'] # use audio(s) from the dataset as reference
ref_dicts = {'spk': reference_audio_path}

reference_embeddings = compute_style(ref_dicts)

spk


In [17]:
# synthesize some texts

texts = [
    "StyleTTS is a style based generative model that can synthesize diverse speech with natural prosody from a reference speech utterance.",
    "Hello friends, speech synthesis is amazing.",
    "Text-to-Speech (TTS) has recently seen great progress in synthesizing high-quality speech owing to the rapid development of parallel TTS systems, but producing speech with naturalistic prosodic variations, speaking styles and emotional tones remains challenging.",
]

In [18]:
converted_samples = {}

for text_id, text in enumerate(texts):
    # tokenize
    ps = global_phonemizer.phonemize([text])
    tokens = textclenaer(ps[0])
    tokens.insert(0, 0)
    tokens.append(0)
    tokens = torch.LongTensor(tokens).to(device).unsqueeze(0)

    with torch.no_grad():
        input_lengths = torch.LongTensor([tokens.shape[-1]]).to(device)
        m = length_to_mask(input_lengths).to(device)
        t_en = model.text_encoder(tokens, input_lengths, m)

        for key, (ref, _) in reference_embeddings.items():

            s = ref.squeeze(1)
            style = s

            d = model.predictor.text_encoder(t_en, style, input_lengths, m)

            x, _ = model.predictor.lstm(d)
            duration = model.predictor.duration_proj(x)
            pred_dur = torch.round(duration.squeeze()).clamp(min=1)

            pred_aln_trg = torch.zeros(input_lengths, int(pred_dur.sum().data))
            c_frame = 0
            for i in range(pred_aln_trg.size(0)):
                pred_aln_trg[i, c_frame:c_frame + int(pred_dur[i].data)] = 1
                c_frame += int(pred_dur[i].data)

            # encode prosody
            en = (d.transpose(-1, -2) @ pred_aln_trg.unsqueeze(0).to(device))
            style = s.expand(en.shape[0], en.shape[1], -1)

            F0_pred, N_pred = model.predictor.F0Ntrain(en, s)

            out = model.decoder((t_en @ pred_aln_trg.unsqueeze(0).to(device)),
                                    F0_pred, N_pred, ref.squeeze().unsqueeze(0))


            c = out.squeeze()
            y_g_hat = generator(c.unsqueeze(0))
            y_out = y_g_hat.squeeze().cpu().numpy()
            if text_id == 0:
                converted_samples[key] = {}
            converted_samples[key][text_id] = y_out

In [19]:
import IPython.display as ipd
for key, waves in converted_samples.items():
    for wave in waves.values():
        print('Synthesized: %s' % key)
        display(ipd.Audio(wave, rate=24000))
    try:
        print('Reference: %s' % key)
        display(ipd.Audio(reference_embeddings[key][-1], rate=24000))
    except:
        continue

Synthesized: spk


Synthesized: spk


Synthesized: spk


Reference: spk


In [ ]:
#the resulted wavs placed here https://github.com/Xrehman/StyleTTS/tree/main/TTS-Assignment/Output_wavs